# 1 线性回归
## 1.1 正态方程

In [1]:
import numpy as np 
X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1)

X_b = np.c_[np.ones((100, 1)), X]

theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y) # 由于np.linalg.inv只能处理方阵
theta_best

array([[4.09638288],
       [2.90297259]])

In [2]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X,y)
print('W:%s,b:%s.'%(lr.intercept_,lr.coef_))

W:[4.09638288],b:[[2.90297259]].


## 1.2 梯度下降
- 幸运的是线性回归模型的均方差损失函数是一个凸函数，这意味着如果你选择曲线上的任意两点，它们的连线段不会与曲线发生交叉（译者注：该线段不会与曲线有第三个交点）。这意味着这个损失函数没有局部最小值，仅仅只有一个全局最小值。同时它也是一个斜率不能突变的连续函数。这两个因素导致了一个好的结果：梯度下降可以无限接近全局最小值。（只要你训练时间足够长，同时学习率不是太大 ）。
- 当我们使用梯度下降的时候，应该确保所有的特征有着相近的尺度范围（例如：使用 Scikit Learn 的 StandardScaler类），否则它将需要很长的时间才能够收敛。

In [10]:
n = 100
eta = 0.1
n_iterations = 1000
theta = np.random.randn(2,1) # 随机初始化系数矩阵:具有标准正态分布的行*列矩阵
for iteration in range(n_iterations):
    loss = 0.5 * 1/n * sum((X_b.dot(theta) - y) **2)
    if iteration % 100 == 0:
        print('iteration:%s,loss:%s,W:%s,b:%s.' % (iteration,loss.round(2),theta[0].round(2),theta[1].round(2)))
    gradient = 1/n * (X_b.T.dot((X_b.dot(theta) - y)))
    theta = theta - eta * gradient

iteration:0,loss:[45.99],W:[-2.03],b:[-0.51].
iteration:100,loss:[0.55],W:[3.74],b:[3.22].
iteration:200,loss:[0.53],W:[4.02],b:[2.97].
iteration:300,loss:[0.53],W:[4.08],b:[2.92].
iteration:400,loss:[0.53],W:[4.09],b:[2.91].
iteration:500,loss:[0.53],W:[4.1],b:[2.9].
iteration:600,loss:[0.53],W:[4.1],b:[2.9].
iteration:700,loss:[0.53],W:[4.1],b:[2.9].
iteration:800,loss:[0.53],W:[4.1],b:[2.9].
iteration:900,loss:[0.53],W:[4.1],b:[2.9].


## 1.3 随机梯度下降
- 每步的梯度计算上只随机选取训练集中的一个样本，损失函数会忽高忽低，大体上呈下降趋势。随着时间的推移，它会非常的靠近最小值，但是它不会停止在一个值上，它会一直在这个值附近摆动，因此，当算法停止的时候，最后的参数还不错，但不是最优值。
- 随机梯度下降算法能够跳过局部最小值。因此，随机梯度下降在寻找全局最小值上比批量梯度下降表现要好。
- 虽然随机性可以很好的跳过局部最优值，但同时它却不能达到最小值，解决这个难题的一个办法是逐渐降低学习率。 开始时，走的每一步较大，然后变得越来越小，从而使算法到达全局最小值。 这个过程被称为模拟退火。

In [20]:
n = 100
n_epochs = 50
theta = np.random.randn(2,1)

def learning_schedule(step):
    return 5/(50 + step)

for epoch in range(n_epochs):
    for i in range(n):
        random_inx = np.random.randint(0,n+1)
        X_b_sample = X_b[random_inx:random_inx+1]
        y_sample = y[random_inx:random_inx+1]
        gradient = X_b_sample.T.dot((X_b_sample.dot(theta) - y_sample)) # 每次只选取1条记录
        theta = theta - learning_schedule(epoch * n + i) * gradient # learning_schedule 替代 eta
        loss = 0.5 * sum((X_b_sample.dot(theta) - y_sample) **2)
    if epoch % 5 == 0:
        print('epoch:%s,loss:%s,W:%s,b:%s.' % (epoch,loss.round(2),theta[0].round(2),theta[1].round(2)))

epoch:0,loss:[0.01],W:[3.73],b:[3.21].
epoch:5,loss:[0.03],W:[3.89],b:[3.1].
epoch:10,loss:[0.34],W:[4.],b:[3.08].
epoch:15,loss:[0.81],W:[3.94],b:[3.01].
epoch:20,loss:[0.35],W:[3.99],b:[3.02].
epoch:25,loss:[0.34],W:[3.95],b:[2.95].
epoch:30,loss:[0.07],W:[4.],b:[3.].
epoch:35,loss:[0.06],W:[4.04],b:[2.97].
epoch:40,loss:[0.16],W:[4.05],b:[2.94].
epoch:45,loss:[0.03],W:[4.07],b:[2.95].


由于每个实例的选择是随机的，有的实例可能在每一代中都被选到，这样其他的实例也可能一直不被选到。如果你想保证每一代迭代过程，算法可以遍历所有实例，一种方法是将训练集打乱重排，然后选择一个实例，之后再继续打乱重排，以此类推一直进行下去。但是这样收敛速度会非常的慢。

**Scikit-Learn** 线性回归的随机梯度下降:

In [32]:
from sklearn.linear_model import SGDRegressor
sgd_reg = SGDRegressor(max_iter=50,
                       learning_rate='constant', # constant，optimal，invscaling
                       eta0=0.1,
                       penalty=None, # 正则项 
                       random_state=7)
sgd_reg.fit(X,y.ravel()) # y 转换成 1*n
print('W:%s,b:%s.'%(sgd_reg.intercept_,sgd_reg.coef_))

W:[4.19072373],b:[2.78582493].


## 1.4 小批量梯度下降